In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [2]:
opts = Options()
# make the driver headless
opts.headless = True
driver = webdriver.Chrome('./chromedriver',options=opts)

In [7]:
# set url of the pages
url = 'https://www.edx.org/search?tab=course'

# use the driver to locate next page button and count how many pages are there
data = []
driver.get(url)
time.sleep(10)
num_pages = int(driver.find_element_by_xpath('//*[@id="displayed-results"]/div[1]/div/nav/ul/li[6]/button').text)
print('Total page: ' + str(num_pages))
# scrape link for each course
for j in range(num_pages):
    print('Page: ' + str(j + 1))
    if j > 0:
        driver.find_element_by_xpath("//*[@id=\"displayed-results\"]/div[1]/div/nav/ul/li[7]/button").click()
        time.sleep(5)
    else:
        time.sleep(0)

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "html.parser")
    summary = soup.find("div",{'static-card-list d-flex m-xl-0 flex-wrap'})
    for i in summary:
        link = [u['href'] for u in i.find_all('a', href=True)]
        data.append([link])
    df = pd.DataFrame(data, columns=['link'])

# clean urls
for i in range(len(df['link'])):
    df['link'][i] = str(df['link'][i]).replace('[','').replace("'",'').replace(']','')
    df['link'][i] = 'https://www.edx.org' + str(df['link'][i])

# df.to_csv('edx_links.csv', sep = ',',encoding='utf-8-sig')
df.head()

Total page: 42
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41
Page: 42


,link
0,https://www.edx.org/course/how-to-learn-online
1,https://www.edx.org/course/programming-for-eve...
2,https://www.edx.org/course/cs50s-introduction-...
3,https://www.edx.org/course/introduction-to-pro...
4,https://www.edx.org/course/marketing-analytics...


In [8]:
# now scrape data for each course using the list of urls
linksdf = pd.read_csv('edx_links.csv')

url = ''
urls = []

#num_pages = len(linksdf['link'])
num_pages = 5
for i in range(num_pages):
    url_page = linksdf['link'][i]
    urls.append(url_page)

# scrape data for each course
# if the element did not exist, use try/except to avoid errors
data = []
for url in urls:
    print(url)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "html.parser")

    # title
    try:
        title = soup.title.text
    except:
        title = 'nil'
    
    # author
    try:
        author = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[4]/div[2]/a").text
    except:
        author = 'nil'
    
    # description
    try:
        desc = soup.find('div', {'class': 'course-description'}).findNext('p').text
    except:
        desc = 'nil'
    
    # subject
    try:
        subject = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[5]/div[2]/a").text
    except:
        subject = 'nil'
    
    # price
    try:
        price = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[3]/div[2]/p/span[1]").text
    except:
        price = '0'
    
    # certificate
    try:
        cert = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[3]/div[2]/p/span[2]").text
    except:
        cert = 'nil'
    
    # enrollment
    try:
        enroll = driver.find_element_by_xpath("//*[@id=\"js-number-enrolled-label\"]/span/span").text
    except:
        enroll = 'nil'
    
    # length
    try:
        length = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[1]/div[2]/span").text
    except:
        length = 'nil'
    
    # difficulty
    try:
        diff = driver.find_element_by_xpath("//*[@id=\"main-content\"]/div/div/div/div[1]/div/div[1]/ul/li[6]/div[2]").text
    except:
        diff = 'nil'
    
    # img link
    try:
        img_link = [u['src'] for u in soup.find_all('img', src=True)][1]
    except:
        img_link = 'nil'
    
    data.append([title, author, subject, desc, price, cert, enroll, length, diff, url, img_link])

# create dataframe from scraped data
df = pd.DataFrame(data, columns=['title', 'author', 'subject', 'desc', 'price', 'certificate_price', 'enrollment', 'length', 'difficulty', 'link', 'img_link'])

# delete unneccesary infomation from the table
df['title'] = df['title'].str.replace("edX","")
df['title'] = df['title'].str.replace("|","")

# if the price was free, replace it with 0
df['price'] = df['price'].str.replace("FREE","0")

# export as csv
df.to_csv('edx_courses.csv', sep = ',',encoding='utf-8-sig')
df.head()

https://www.edx.org/professional-certificate/berkeleyx-science-of-happiness-at-work
https://www.edx.org/micromasters/hecmontrealx-ux-design-and-evaluation
https://www.edx.org/micromasters/purduex-quantum-technology-computing
https://www.edx.org/micromasters/writing-for-performance-and-the-entertainment-industries
https://www.edx.org/micromasters/doanex-sustainable-agribusiness


,title,author,subject,desc,price,certificate_price,enrollment,length,difficulty,link,img_link
0,The Science of Happiness at Work Professional ...,nil,nil,nil,0,nil,nil,nil,nil,https://www.edx.org/professional-certificate/b...,/images/experiments/ProgramDetails/business-ma...
1,UX Design and Evaluation MicroMasters® Program,nil,nil,nil,0,nil,nil,nil,nil,https://www.edx.org/micromasters/hecmontrealx-...,/images/experiments/ProgramDetails/data-scienc...
2,Quantum Technology: Computing MicroMasters® Pr...,nil,nil,nil,0,nil,nil,nil,nil,https://www.edx.org/micromasters/purduex-quant...,/images/experiments/ProgramDetails/data-scienc...
3,Writing for Performance and the Entertainment ...,nil,nil,nil,0,nil,nil,nil,nil,https://www.edx.org/micromasters/writing-for-p...,/images/experiments/ProgramDetails/data-scienc...
4,Sustainable Agribusiness MicroMasters® Program,nil,nil,nil,0,nil,nil,nil,nil,https://www.edx.org/micromasters/doanex-sustai...,/images/experiments/ProgramDetails/business-ma...


In [195]:
# data cleansing
# drop price, link and img_link columns
clean_df = pd.read_csv("./edx_courses.csv", sep=",")
clean_df = clean_df.drop(['Unnamed: 0', 'price', 'link', 'img_link'], axis=1)
clean_df.head()

,title,author,subject,desc,certificate_price,enrollment,length,difficulty
0,How to Learn Online,edX,Education & Teacher Training,"Designed for those who are new to elearning, t...",Add a Verified Certificate for $49 USD,"134,568",2 Weeks,Introductory
1,Programming for Everybody (Getting Started wit...,MichiganX,Computer Science,This course aims to teach everyone the basics ...,Add a Verified Certificate for $49 USD,"311,894",7 Weeks,Introductory
2,CS50's Introduction to Computer Science,HarvardX,Computer Science,"This is CS50x , Harvard University's introduct...",Add a Verified Certificate for $199 USD,"2,503,137",12 Weeks,Introductory
3,Introduction to Project Management,AdelaideX,Business & Management,Project management is an essential skill-set f...,Add a Verified Certificate for $199 USD,"309,697",6 Weeks,Introductory
4,Marketing Analytics: Marketing Measurement Str...,BerkeleyX,Computer Science,Begin your journey in a new career in marketin...,Add a Verified Certificate for $249 USD,"84,939",4 Weeks,Introductory


In [196]:
# only get numerical value from 'certificate_price' column
for i in range(len(clean_df['certificate_price'])):
    string = str(clean_df['certificate_price'][i])
    tmp = re.findall(r'\d+', string)
    val = list(map(int, tmp))
    clean_df['certificate_price'][i] = str(val)

clean_df['certificate_price'] = clean_df['certificate_price'].str.replace("[","").str.replace("]","")
clean_df['certificate_price'] = clean_df['certificate_price'].str.replace(",",".").str.replace(" ","")

for i in range(len(clean_df['certificate_price'])):
    if clean_df['certificate_price'][i] == "":
        clean_df['certificate_price'][i] = 0

clean_df.head()

,title,author,subject,desc,certificate_price,enrollment,length,difficulty
0,How to Learn Online,edX,Education & Teacher Training,"Designed for those who are new to elearning, t...",49,"134,568",2 Weeks,Introductory
1,Programming for Everybody (Getting Started wit...,MichiganX,Computer Science,This course aims to teach everyone the basics ...,49,"311,894",7 Weeks,Introductory
2,CS50's Introduction to Computer Science,HarvardX,Computer Science,"This is CS50x , Harvard University's introduct...",199,"2,503,137",12 Weeks,Introductory
3,Introduction to Project Management,AdelaideX,Business & Management,Project management is an essential skill-set f...,199,"309,697",6 Weeks,Introductory
4,Marketing Analytics: Marketing Measurement Str...,BerkeleyX,Computer Science,Begin your journey in a new career in marketin...,249,"84,939",4 Weeks,Introductory


In [199]:
# convert 'certificate_price' dtype to float
clean_df['certificate_price'] = clean_df['certificate_price'].astype(float)
clean_df.dtypes

title                 object
author                object
subject               object
desc                  object
certificate_price    float64
enrollment            object
length                object
difficulty            object
dtype: object

In [201]:
# prepare 'enrollment', 'length_by_week', and 'difficulty' columns for converting to numeric values
clean_df['enrollment'] = clean_df['enrollment'].str.replace(",","").str.replace("nil","0")
clean_df['length'] = clean_df['length'].str.replace(" Weeks","")
clean_df['difficulty'] = clean_df['difficulty'].str.replace("English","Introductory")
# rename 'length' to 'length_by_week'
clean_df = clean_df.rename(columns={'length': 'length_by_week'})
clean_df.head()

,title,author,subject,desc,certificate_price,enrollment,length_by_week,difficulty
0,How to Learn Online,edX,Education & Teacher Training,"Designed for those who are new to elearning, t...",49.00,134568,2,Introductory
1,Programming for Everybody (Getting Started wit...,MichiganX,Computer Science,This course aims to teach everyone the basics ...,49.00,311894,7,Introductory
2,CS50's Introduction to Computer Science,HarvardX,Computer Science,"This is CS50x , Harvard University's introduct...",199.00,2503137,12,Introductory
3,Introduction to Project Management,AdelaideX,Business & Management,Project management is an essential skill-set f...,199.00,309697,6,Introductory
4,Marketing Analytics: Marketing Measurement Str...,BerkeleyX,Computer Science,Begin your journey in a new career in marketin...,249.00,84939,4,Introductory
...,...,...,...,...,...,...,...,...
995,Exploring Psychology's Core Concepts走进心理学,TsinghuaX,Social Sciences,This course provides an introduction to the ba...,139.00,5903,6,Introductory
996,AP® Calculus: Challenging Concepts from Calcul...,DavidsonNext,Math,Well-respected AP instructors from around the ...,49.00,30347,14,Introductory
997,Thinking & Acting like an Entrepreneur,RWTHx,Business & Management,All of today’s well-known companies got starte...,199.00,16767,6,Intermediate
998,"Cannabis Compliance, Testing, and Research",DoaneX,Environmental Studies,Learn compliance and testing methods related t...,179.00,0,3,Introductory


In [202]:
# convert 'enrollment' and 'length_by_week' dtypes to int
clean_df['enrollment'] = clean_df['enrollment'].astype(int)
clean_df['length_by_week'] = clean_df['length_by_week'].astype(int)
clean_df.dtypes

title                 object
author                object
subject               object
desc                  object
certificate_price    float64
enrollment             int64
length_by_week         int64
difficulty            object
dtype: object

In [203]:
# now prepare 'difficulty' column for data manipulation
clean_df['difficulty'].unique()

array(['Introductory', 'Intermediate', 'Advanced'], dtype=object)

In [204]:
# assign 0 for Introductory, 1 for Intermediate, and 2 for Advanced
clean_df['difficulty'] = clean_df['difficulty'].str.replace("Introductory","0").str.replace("Intermediate","1").str.replace("Advanced","2")
clean_df['difficulty'].unique()

array(['0', '1', '2'], dtype=object)

In [205]:
# convert 'difficulty' dtype to int
clean_df['difficulty'] = clean_df['difficulty'].astype(int)

In [206]:
clean_df.dtypes

title                 object
author                object
subject               object
desc                  object
certificate_price    float64
enrollment             int64
length_by_week         int64
difficulty             int64
dtype: object

In [208]:
# export as csv
clean_df.to_csv('edx_free_courses_cleaned.csv', sep = ',',encoding='utf-8-sig', float_format='%.2f')